In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from enum import Enum
import datetime
import gym
from gym import spaces
from dqn import DQN
from utils import tradeclass
from utils import BCTrade

from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.core.environment import Environment

In [2]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
    
df = pd.read_csv('bitbayEUR.csv',names=['date','price','volume'], parse_dates=['date'], date_parser=dateparse,index_col=0)

df['ask'] = df['price']*1.000
df['bid'] = df['price']*1.000
df = df.reindex(columns=['price','ask','bid','volume'])

df

,price,ask,bid,volume
date,,,,
2015-03-17 18:27:49,290.00,290.00,290.00,3.045430
2015-03-18 13:31:32,290.00,290.00,290.00,0.042517
2015-03-19 10:12:58,290.00,290.00,290.00,2.521450
2015-03-19 10:20:03,290.00,290.00,290.00,0.625478
2015-03-19 10:20:23,290.00,290.00,290.00,4.521450
...,...,...,...,...
2018-07-27 21:48:25,7049.04,7049.04,7049.04,0.001698
2018-07-27 21:49:15,7049.04,7049.04,7049.04,0.200000
2018-07-27 21:49:15,7049.05,7049.05,7049.05,0.001698


In [3]:
#df['prev'] = df['price'].rolling(2).apply(lambda x: (x[1]/x[0])-1 ,raw=True)
    
#df['ma100']=df['price']/df['price'].rolling(10).mean()-1
df['ma10001']=df['price']/df['price'].rolling('60s').mean()-1
#df['ma10002']=df['price']/df['price'].rolling('60s').max()-1
#df['ma10003']=df['price']/df['price'].rolling('60s').min()-1

df['ma1000x1']=df['price']/df['price'].rolling('1h').mean()-1
#df['ma1000x2']=df['price']/df['price'].rolling('1h').max()-1
#df['ma1000x3']=df['price']/df['price'].rolling('1h').min()-1

df['ma1000y1']=df['price']/df['price'].rolling('10h').mean()-1
#df['ma1000y2']=df['price']/df['price'].rolling('10h').max()-1
#df['ma1000y3']=df['price']/df['price'].rolling('10h').min()-1

df['ma100001']=df['price']/df['price'].rolling('1D').mean()-1
#df['ma100002']=df['price']/df['price'].rolling('1D').max()-1
#df['ma100003']=df['price']/df['price'].rolling('1D').min()-1
df['ma1']=df['price']/df['price'].rolling('10D').mean()-1
#df['ma2']=df['price']/df['price'].rolling('10D').max()-1
#df['ma3']=df['price']/df['price'].rolling('10D').min()-1
df['mafff1']=df['price']/df['price'].rolling('20D').mean()-1
#df['mafff2']=df['price']/df['price'].rolling('20D').max()-1
#df['mafff3']=df['price']/df['price'].rolling('20D').min()-1
df = df.dropna()
#df = df.drop('price', 1)
#df = df.drop('volume', 1)
df.columns

Index(['price', 'ask', 'bid', 'volume', 'ma10001', 'ma1000x1', 'ma1000y1',
       'ma100001', 'ma1', 'mafff1'],
      dtype='object')

In [4]:
env =BCTrade(df)
env.render()
from ray.tune.registry import register_env
from ray.rllib.agents import ppo

def env_creator(env_config):
    return BCTrade(df)

register_env("my_env", env_creator)
#trainer = ppo.PPOTrainer(env="my_env")

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


1000.0


In [13]:
import ray
from ray import tune

#ray.init()
tune.run(
    "PPO",
    stop={"training_iteration": 200,"episode_reward_mean": 100},
    checkpoint_at_end=True,
    config={
        #"env": "CartPole-v0",
        "env": "my_env",
        "num_gpus": 0,
        "num_workers": 1,
        #"keep_checkpoints_num":1,
       
        "lr": tune.grid_search([0.01]),
        #"lr": tune.grid_search([0.01, 0.001, 0.0001]),
    },
)

Trial name,status,loc,lr
PPO_my_env_def32_00000,PENDING,,0.01


(pid=7053) 2021-12-27 20:23:41,113	INFO trainer.py:753 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=7053) 2021-12-27 20:23:41,113	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=7053) 2021-12-27 20:23:41,113	INFO trainer.py:772 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=12336) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=12336)   "Box bound precision lowered by casting to {}".format(self.dtype)
(pid=12336) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) wi

Trial name,status,loc,lr
PPO_my_env_def32_00000,RUNNING,10.0.0.4:7053,0.01


(pid=7053) 2021-12-27 20:23:48,200	WARNING trainer_template.py:186 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!
(pid=7053) 2021-12-27 20:23:48,230	WARNING util.py:57 -- Install gputil for GPU system monitoring.


Trial name,status,loc,lr
PPO_my_env_def32_00000,RUNNING,10.0.0.4:7053,0.01


...

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_def32_00000,RUNNING,10.0.0.4:7053,0.01,57,890.825,228000,93.5179,1862.02,-259.795,1333.89


Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_def32_00000,RUNNING,10.0.0.4:7053,0.01,57,890.825,228000,93.5179,1862.02,-259.795,1333.89


Result for PPO_my_env_def32_00000:
  agent_timesteps_total: 232000
  custom_metrics: {}
  date: 2021-12-27_20-38-56
  done: true
  episode_len_mean: 1389.25
  episode_media: {}
  episode_reward_max: 1862.0179458307396
  episode_reward_mean: 102.25327908165323
  episode_reward_min: -259.79473345372253
  episodes_this_iter: 1
  episodes_total: 8198
  experiment_id: c31d2b406cd044b89e31b40c053cbbf8
  hostname: rnikolaus1
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 6.041784850197018e-09
          cur_lr: 0.009999999776482582
          entropy: 8.475417589970391e-10
          entropy_coeff: 0.0
          kl: 2.5537508305945034e-12
          model: {}
          policy_loss: -0.008577629923820496
          total_loss: 23.483259201049805
          vf_explained_var: -4.999099356695069e-09
          vf_loss: 23.491836547851562
    num_agent_steps_sampled: 232000
    num_agent_steps_trained: 232000
    num_steps_sampled: 232

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_def32_00000,TERMINATED,10.0.0.4:7053,0.01,58,906.162,232000,102.253,1862.02,-259.795,1389.25


2021-12-27 20:38:56,911	INFO tune.py:630 -- Total run time: 920.21 seconds (919.85 seconds for the tuning loop).
